In [ ]:
import numpy as np
from astropy import units as u, constants as const
from matplotlib import pyplot as plt
from scipy.stats import beta
from scipy.special import beta as betafct
from numba import jit, jitclass
from scipy import interpolate
import logging
import h5py
from types import SimpleNamespace
from functools import partial
import inspect 
import re

import sys
import os
sys.path.insert(1, '/home/jsipple/one_d_spherical_collapse/one-d-spherical-collapse-of-fuzzy-dark-matter/src')

import importlib
import simulation_strategies, collapse, plotting, utils
importlib.reload(simulation_strategies)
importlib.reload(collapse)
importlib.reload(plotting)
importlib.reload(utils)
from simulation_strategies import *
from collapse import *
from plotting import *
from utils import *

In [ ]:
def my_mpl():
    plt.rc('font', family='serif', size=20)
    plt.rc('axes', grid=True)
    plt.rc('lines', lw=3)
    ts = 8
    plt.rc('xtick.minor', size=ts-2)
    plt.rc('ytick.minor', size=ts-2)
    plt.rc('xtick.major', size=ts)
    plt.rc('ytick.major', size=ts)
    plt.rc('figure', figsize=[12, 9])
my_mpl()
logging.basicConfig(level=logging.DEBUG,
                    format='%(asctime)s - %(levelname)s - %(message)s')
logger = logging.getLogger(__name__)
logging.getLogger('matplotlib').setLevel(logging.ERROR)

In [ ]:
t_i = 1
G=1
rho = 1/(6*np.pi*G*t_i**2)
H = 2/(3*t_i)

In [ ]:
import cProfile
import pstats
# 't_max': 5.7,
# Create a SphericalCollapse instance
config = {
    'softlen': 0,
    'density_strategy': "const",
    'point_mass': 1,
    'safety_factor': 1e-4,
    'gamma': 0,
    'thickness_coef': 0,
    'j_coef': 7e-1,
    'angular_momentum_strategy': 'const',
    't_max': 3,
    'dt_min': 1e-8,
    'N': 1,
    'r_min': 1e-2,
    'H': 0,
    'r_min': 1e-2,
    'r_max': 1,
    'm_tot': 1, 
    'stepper_strategy': 'beeman',
    'timescale_strategy': 'dyn_rmin',
    'm_enc_strategy': 'const_exclusive'
}
x = SphericalCollapse(config)

# Profile the run method
profiler = cProfile.Profile()
profiler.enable()
results = x.run()
profiler.disable()

# Print the profiling results
stats = pstats.Stats(profiler).sort_stats('cumulative')
stats.print_stats(20)  # Print top 20 time-consuming functions

In [ ]:
G = 1
m = 1
M = 2
J = 0.7
E_tot = -G*M*m + 0.5*m*J**2
a = -E_tot
b = -G*M*m
c = 0.5*m*J**2
r_close = (-b - np.sqrt(b**2 - 4*a*c)) / (2*a)
print(r_close)

In [ ]:
min(results['r'])

In [ ]:
def plot_global_property(results, property_name, limit_axis=True, ylim=None):
    """
    Plot the progress of a global property over time.
    
    Parameters:
    - results: dict, output from SphericalCollapse.run()
    - property_name: str, name of the property to plot ('t_dyn', 'e_tot', etc.)
    - limit_axis: bool, whether to limit the y-axis to exclude extreme values
    - ylim: tuple, custom y-axis limits (min, max)
    """
    if property_name not in results:
        raise ValueError(f"Property '{property_name}' not found in results.")
    
    time = results['t']
    property_data = results[property_name]
    
    plt.figure(figsize=(10, 6))
    plt.plot(time, property_data)
    plt.xlabel('Time')
    plt.ylabel(property_name.capitalize())
    plt.title(f'{property_name.capitalize()} vs Time')
    plt.grid(True)
    
    if ylim:
        plt.ylim(ylim)
    elif limit_axis:
        lower_percentile = np.percentile(property_data, 1)
        upper_percentile = np.percentile(property_data, 99)
        plt.ylim(lower_percentile, upper_percentile)
    
    

def plot_shell_property(results, property_name, shell_indices=None, num_shells=5, limit_axis=True, ylim=None, yscale='linear', title=None, vs_r=False):
    """
    Plot the progress of selected shells over time for a given property.
    
    Parameters:
    - results: dict, output from SphericalCollapse.run()
    - property_name: str, name of the property to plot ('r', 'v', 'a', 'e_tot', 'e_g', 'e_k', 'e_r')
    - shell_indices: list of int, indices of shells to plot. If None, evenly spaced shells will be selected.
    - num_shells: int, number of shells to plot if shell_indices is None
    - limit_axis: bool, whether to limit the y-axis to exclude extreme values
    - ylim: tuple, custom y-axis limits (min, max)
    """
    if property_name not in results:
        raise ValueError(f"Property '{property_name}' not found in results.")
    
    time = results['t']
    property_data = results[property_name]
    
    if property_data.ndim == 1:
        # If the property is a global quantity, use plot_global_property instead
        return plot_global_property(results, property_name, limit_axis, ylim)
    
    if shell_indices is None:
        total_shells = property_data.shape[1]
        shell_indices = np.linspace(0, total_shells-1, min(total_shells, num_shells), dtype=int)
    
    plt.figure(figsize=(10, 6))
    for idx in shell_indices:
        if vs_r:
            plt.plot(results['r'][:, idx], property_data[:, idx], label=f'Shell {idx+1}')
        else:
            plt.plot(time, property_data[:, idx], label=f'Shell {idx+1}')
    
    plt.yscale(yscale)
    plt.xlabel('Time' if not vs_r else 'Radius')
    plt.ylabel(property_name.capitalize())
    if title is None:
        plt.title(f'{property_name}')
    else:
        plt.title(title)
    plt.legend()
    plt.grid(True)
    
    if ylim:
        plt.ylim(ylim)
    elif limit_axis:
        data_to_plot = property_data[:, shell_indices]
        lower_percentile = 0.9*np.percentile(data_to_plot, 1)
        upper_percentile = 1.1*np.percentile(data_to_plot, 99)
        plt.ylim(lower_percentile, upper_percentile)
    
    

def plot_energy_components(results, shell_index=None, limit_axis=True, ylim=None):
    """
    Plot the progress of different energy components over time.
    If shell_index is provided, plot for a specific shell, otherwise plot global energy.
    
    Parameters:
    - results: dict, output from SphericalCollapse.run()
    - shell_index: int or None, index of the shell to plot. If None, plot global energy.
    - limit_axis: bool, whether to limit the y-axis to exclude extreme values
    - ylim: tuple, custom y-axis limits (min, max)
    """
    time = results['t']
    energy_components = ['e_tot', 'e_g', 'e_k', 'e_r']
    
    plt.figure(figsize=(10, 6))
    all_energy_data = []
    for component in energy_components:
        if component in results:
            energy_data = results[component]
            if energy_data.ndim > 1 and shell_index is not None:
                energy_data = energy_data[:, shell_index]
            plt.plot(time, energy_data, label=component.capitalize())
            all_energy_data.extend(energy_data)
    
    plt.xlabel('Time')
    plt.ylabel('Energy')
    title = 'Energy Components vs Time'
    if shell_index is not None:
        title += f' for Shell {shell_index}'
    plt.title(title)
    plt.legend()
    plt.grid(True)
    
    if ylim:
        plt.ylim(ylim)
    elif limit_axis:
        lower_percentile = np.percentile(all_energy_data, 1)
        upper_percentile = np.percentile(all_energy_data, 99)
        lower_limit = lower_percentile * 0.9 if lower_percentile > 0 else lower_percentile * 1.1
        upper_limit = upper_percentile * 1.1 if upper_percentile > 0 else upper_percentile * 0.9
        plt.ylim(lower_limit, upper_limit)
    
    

def plot_timescales(results, shell_index=None, limit_axis=False, ylim=None):
    """
    Plot the progress of different timescale components over time.
    If shell_index is provided, plot for a specific shell, otherwise plot global timescales.
    
    Parameters:
    - results: dict, output from SphericalCollapse.run()
    - shell_index: int or None, index of the shell to plot. If None, plot global timescales.
    - limit_axis: bool, whether to limit the y-axis to exclude extreme values
    - ylim: tuple, custom y-axis limits (min, max)
    """
    time = results['t']
    timescale_components = ['t_dyn', 't_vel', 't_acc', 't_cross', 't_zero', 't_rmin', 't_rmina', 'dt']
    
    plt.figure(figsize=(10, 6))
    all_timescale_data = []
    for component in timescale_components:
        if component in results and not all(x is None for x in results[component]):
            timescale_data = results[component]
            if timescale_data.ndim > 1 and shell_index is not None:
                timescale_data = timescale_data[:, shell_index]
            plt.plot(time, timescale_data, label=component)
            all_timescale_data.extend(timescale_data)
    
    plt.xlabel('Time')
    plt.ylabel('Timescale')
    title = 'Timescale Components vs Time'
    if shell_index is not None:
        title += f' for Shell {shell_index}'
    plt.title(title)
    plt.legend()
    plt.grid(True)
    
    plt.yscale('log')  # Set y-axis to logarithmic scale
    
    if ylim:
        plt.ylim(ylim)
    elif limit_axis:
        lower_percentile = np.percentile(all_timescale_data, 1)
        upper_percentile = np.percentile(all_timescale_data, 99)
        plt.ylim(lower_percentile, upper_percentile)
    
    

def plot_total_energy_relative_change(results, limit_axis=True, ylim=None, yscale='linear'):
    """
    Plot the relative change of the sum of total energies of all individual shells over time,
    compared to the initial total energy.
    
    Parameters:
    - results: dict, output from SphericalCollapse.run()
    - limit_axis: bool, whether to limit the y-axis to exclude extreme values
    - ylim: tuple, custom y-axis limits (min, max)
    - yscale: str, scale of y-axis ('linear', 'log', 'symlog', etc.)
    """
    time = results['t']
    e_tot = results['e_tot']
    
    # Calculate the sum of total energies for all shells at each time step
    total_energy_sum = np.sum(e_tot, axis=1)
    
    # Get the initial total energy
    initial_total_energy = total_energy_sum[0]
    
    # Calculate the relative change compared to the initial energy
    relative_change = (total_energy_sum - initial_total_energy) / initial_total_energy
    if yscale == 'log':
        relative_change = np.abs(relative_change)
    
    plt.figure(figsize=(10, 6))
    plt.plot(time, relative_change)
    plt.xlabel('Time')
    plt.ylabel('Relative Change in Total Energy')
    plt.title(r'$\Delta E/E_0$')
    plt.grid(True)
    
    plt.yscale(yscale)  # Set y-axis scale
    
    if ylim:
        plt.ylim(ylim)
    elif limit_axis:
        lower_percentile = np.percentile(relative_change, 1)
        upper_percentile = np.percentile(relative_change, 99)
        lower_limit = lower_percentile * 0.9 if lower_percentile > 0 else lower_percentile * 1.1
        upper_limit = upper_percentile * 1.1 if upper_percentile > 0 else upper_percentile * 0.9
        plt.ylim(lower_limit, upper_limit)

# Example usage:
plot_timescales(results)
#plot_global_property(results, 'e_tot')
shells = None#[0,24,49,74,99]
plot_shell_property(results, 'r', ylim=[-0.1, 2.2], shell_indices=shells)
plot_shell_property(results, 'r', yscale='log', limit_axis=False, shell_indices=shells)
plot_shell_property(results, 'r', vs_r=True, ylim=[-0.1, 2.2], shell_indices=shells)

#plot_global_property(results, 'num_crossing', limit_axis=False)
plot_total_energy_relative_change(results, limit_axis=False, yscale='linear',)
#plot_shell_property(results, 'v')
plot_shell_property(results, 'm_enc', limit_axis=False, shell_indices=shells)
plot_shell_property(results, 'v', limit_axis=False, shell_indices=shells)
plot_shell_property(results, 'a', limit_axis=False, shell_indices=shells)
plot_shell_property(results, 'm_enc', limit_axis=False, shell_indices=shells, vs_r=True)
plot_shell_property(results, 'v', limit_axis=False, shell_indices=shells, vs_r=True)
plot_shell_property(results, 'a', limit_axis=False, shell_indices=shells, vs_r=True)
plot_energy_components(results, shell_index=0, limit_axis=False)  # Plot energy for a specific shell
plot_energy_components(results, shell_index=-1, limit_axis=False)  # Plot energy for a specific shell
#analyze_energy_conservation(results)

def plot_normalized_radius(results, shell_index=None, num_shells=None, limit_axis=False, ylim=None):
    """
    Plot the radius divided by the maximum value r takes for each shell.
    
    Parameters:
    - results: dict, output from SphericalCollapse.run()
    - shell_index: int or list of int, index/indices of the shell(s) to plot. If None, plot all shells.
    - num_shells: int, number of shells to plot starting from the first shell. If None, plot all shells.
    - limit_axis: bool, whether to limit the y-axis to exclude extreme values
    - ylim: tuple, custom y-axis limits (min, max)
    """
    time = results['t']
    r = results['r']
    
    # Calculate the maximum value of r for each shell
    max_r = np.max(r, axis=0)
    
    # Normalize r by the maximum value for each shell
    normalized_r = r / max_r
    
    plt.figure(figsize=(10, 6))
    
    if shell_index is None:
        if num_shells is None:
            shell_indices = range(normalized_r.shape[1])
        else:
            shell_indices = np.linspace(0, normalized_r.shape[1] - 1, num_shells, dtype=int)
    elif isinstance(shell_index, int):
        shell_indices = [shell_index]
    else:
        shell_indices = shell_index
    
    for i in shell_indices:
        plt.plot(time, normalized_r[:, i], label=f'Shell {i}')
    
    plt.xlabel('Time')
    plt.ylabel('Normalized Radius (r / max(r))')
    plt.title('Normalized Radius Over Time')
    plt.legend()
    plt.grid(True)
    
    if ylim:
        plt.ylim(ylim)
    elif limit_axis:
        lower_percentile = np.percentile(normalized_r, 1)
        upper_percentile = np.percentile(normalized_r, 99)
        lower_limit = lower_percentile * 0.9 if lower_percentile > 0 else lower_percentile * 1.1
        upper_limit = upper_percentile * 1.1 if upper_percentile > 0 else upper_percentile * 0.9
        plt.ylim(lower_limit, upper_limit)
    
    

# Example usage:
plot_normalized_radius(results, num_shells=5, limit_axis=False)



In [ ]:
5*np.pi/np.sqrt(8)

In [ ]:
def analyze_energy_conservation(results):
    t = results['t']
    e_tot = np.sum(results['e_tot'], axis=1)
    e_k = np.sum(results['e_k'], axis=1)
    e_g = np.sum(results['e_g'], axis=1)
    e_r = np.sum(results['e_r'], axis=1)
    
    # Calculate relative energy change
    e_rel_change = (e_tot - e_tot[0]) / e_tot[0]
    
    # Calculate energy change per time step
    de_dt = np.diff(e_tot) / np.diff(t)
    
    # Plot results
    fig, axs = plt.subplots(3, 1, figsize=(12, 18), sharex=True)
    
    axs[0].plot(t, e_tot, label='Total')
    axs[0].plot(t, e_k, label='Kinetic')
    axs[0].plot(t, e_g, label='Gravitational')
    axs[0].plot(t, e_r, label='Rotational')
    axs[0].set_ylabel('Energy')
    axs[0].legend()
    axs[0].set_title('Energy Components')
    
    axs[1].plot(t, e_rel_change)
    axs[1].set_ylabel('Relative Energy Change')
    axs[1].set_title('Relative Total Energy Change')
    
    axs[2].plot(t[1:], de_dt)
    axs[2].set_ylabel('dE/dt')
    axs[2].set_xlabel('Time')
    axs[2].set_title('Energy Change Rate')
    
    plt.tight_layout()
    
    
    # Identify largest energy changes
    largest_changes = np.argsort(np.abs(de_dt))[-5:]
    print("Time steps with largest energy changes:")
    for i in largest_changes:
        print(f"Time: {t[i+1]:.6f}, dE/dt: {de_dt[i]:.6e}")

# Use the function
analyze_energy_conservation(results)

In [ ]:
from utils import load_simulation_data

# Assuming all the plotting functions are already defined in this notebook

def process_results(results, sim_name):
    print(sim_name)
    #plot_timescales(results)
    shells = None  # [0,24,49,74,99]
    plot_shell_property(results, 'r', ylim=[-0.1, 2], shell_indices=shells, title=sim_name)
    plot_shell_property(results, 'r', yscale='log', limit_axis=False, shell_indices=shells)
    plot_total_energy_relative_change(results, limit_axis=False, yscale='linear')
    plot_total_energy_relative_change(results, limit_axis=False, yscale='log', ylim=[1e-5, None])
    #plot_shell_property(results, 'm_enc', limit_axis=False, shell_indices=shells)
    #plot_shell_property(results, 'v', limit_axis=False, shell_indices=shells)
    #plot_shell_property(results, 'a', limit_axis=False, shell_indices=shells)
    #plot_energy_components(results, shell_index=0, limit_axis=False)
    #plot_energy_components(results, shell_index=-1, limit_axis=False)
    #analyze_energy_conservation(results)
    

# Process all .h5 files in the src/ directory
folder = 'thickness_coef'
src_dir = '/home/jsipple/one_d_spherical_collapse/one-d-spherical-collapse-of-fuzzy-dark-matter/src/runs/'+folder
fn = '/home/jsipple/one_d_spherical_collapse/one-d-spherical-collapse-of-fuzzy-dark-matter/src/runs/sim_N2_soft0_sf0.001_tc0_jc0.h5'
params, results = load_simulation_data(fn)
# Process and display results
process_results(results, 'sim_N2_soft0_sf0.001_tc0_jc0.h5')
for filename in os.listdir(src_dir):
    if filename.endswith('.h5'):
        file_path = os.path.join(src_dir, filename)        
        # Load the simulation data
        params, results = load_simulation_data(file_path)
        
        # Process and display results
        process_results(results, folder+os.path.splitext(filename)[0])
        

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

def cycloid_solution(t, A):
    """
    Compute the cycloid solution for spherical collapse.
    
    Parameters:
    t : array-like
        Time values
    A : float
        Amplitude parameter
    
    Returns:
    r : array-like
        Radial positions
    v : array-like
        Velocities
    """
    theta = np.sqrt(np.float128(8) * np.pi / np.float128(A)) * np.float128(t)
    r = np.float128(0.5) * np.float128(A) * (np.float128(1) + np.cos(theta))
    v = -np.float128(0.5) * np.float128(A) * np.sqrt(np.float128(8) * np.pi / np.float128(A)) * np.sin(theta)
    return r, v

def compute_energies(r, v, m, G):
    """
    Compute kinetic, potential, and total energy.
    
    Parameters:
    r : array-like
        Radial positions
    v : array-like
        Velocities
    m : float
        Mass of the particle
    G : float
        Gravitational constant
    
    Returns:
    E_k : array-like
        Kinetic energy
    E_p : array-like
        Potential energy
    E_tot : array-like
        Total energy
    """
    E_k = np.float128(0.5) * np.float128(m) * v**2
    E_p = -np.float128(G) * np.float128(m)**2 / r
    E_tot = E_k + E_p
    return E_k, E_p, E_tot

# Set up parameters
G = np.float128(1.0)  # Gravitational constant
m = np.float128(1.0)  # Mass of the particle
A = np.float128(1.0)  # Amplitude parameter
t_max = np.float128(2) * np.pi * np.sqrt(np.float128(A) / (np.float128(8) * np.pi))  # One full cycle
t = np.linspace(np.float128(0), t_max, 1000, dtype=np.float128)

# Compute cycloid solution
r, v = cycloid_solution(t, A)

# Compute energies
E_k, E_p, E_tot = compute_energies(r, v, m, G)

# Plot results
plt.figure(figsize=(12, 8))
plt.subplot(211)
plt.plot(t, r, label='r(t)')
plt.plot(t, v, label='v(t)')
plt.xlabel('Time')
plt.ylabel('r, v')
plt.legend()
plt.title('Cycloid Solution')

plt.subplot(212)
plt.plot(t, np.abs(E_k), label='Kinetic Energy')
plt.plot(t, np.abs(E_p), label='Potential Energy')
plt.plot(t, np.abs(E_tot), label='Total Energy')
plt.xlabel('Time')
plt.ylabel('Energy')
plt.yscale('log')
plt.ylim(1e0, 1e3)
plt.legend()
plt.title('Energy Conservation')

plt.tight_layout()


# Print energy conservation statistics
E_tot_initial = E_tot[0]
E_tot_relative_error = (E_tot - E_tot_initial) / E_tot_initial
print(f"Maximum relative error in total energy: {np.max(np.abs(E_tot_relative_error)):.2e}")
print(f"Standard deviation of relative error in total energy: {np.std(E_tot_relative_error):.2e}")

In [ ]:
# Define the gravitational acceleration function
def gravitational_acceleration(r, G, M):
    return -G * M / r**2

# Velocity Verlet integrator with reflecting sphere
def velocity_verlet(r0, v0, dt, steps, G, M, r_min):
    r, v = np.zeros(steps), np.zeros(steps)
    r[0], v[0] = r0, v0
    
    for i in range(1, steps):
        a = gravitational_acceleration(r[i-1], G, M)
        r[i] = r[i-1] + v[i-1]*dt + 0.5*a*dt**2
        
        # Reflect off the sphere at r_min
        if r[i] < r_min:
            r[i] = 2*r_min - r[i]
            v[i-1] = -v[i-1]  # Reverse velocity for elastic collision
        
        a_next = gravitational_acceleration(r[i], G, M)
        v[i] = v[i-1] + 0.5*(a + a_next)*dt
    
    return r, v

# Beeman integrator with reflecting sphere
def beeman(r0, v0, dt, steps, G, M, r_min):
    r, v = np.zeros(steps), np.zeros(steps)
    r[0], v[0] = r0, v0
    a_prev = gravitational_acceleration(r0, G, M)
    a = a_prev
    
    for i in range(1, steps):
        r[i] = r[i-1] + v[i-1]*dt + (4*a - a_prev)*dt**2/6
        
        # Reflect off the sphere at r_min
        if r[i] < r_min:
            r[i] = 2*r_min - r[i]
            v[i-1] = -v[i-1]  # Reverse velocity for elastic collision
        
        a_next = gravitational_acceleration(r[i], G, M)
        v[i] = v[i-1] + (2*a_next + 5*a - a_prev)*dt/6
        a_prev, a = a, a_next
    
    return r, v

# Set up parameters for the test
G = 1.0
M = 1.0
r0 = 1.0
v0 = 0.5
dt = 0.0001
steps = 100000  # Increased number of steps to see multiple bounces
r_min = 0.1  # Radius of the reflecting sphere
t = np.arange(steps) * dt

# Run simulations
r_vv, v_vv = velocity_verlet(r0, v0, dt, steps, G, M, r_min)
r_beeman, v_beeman = beeman(r0, v0, dt, steps, G, M, r_min)

# Calculate energies
E_vv = 0.5 * v_vv**2 - G*M/r_vv
E_beeman = 0.5 * v_beeman**2 - G*M/r_beeman

# Plot results
plt.figure(figsize=(15, 10))

plt.subplot(221)
plt.plot(t, r_vv, label='Velocity Verlet')
plt.plot(t, r_beeman, label='Beeman')
plt.axhline(y=r_min, color='r', linestyle='--', label='Reflecting Sphere')
plt.xlabel('Time')
plt.ylabel('Radius')
plt.legend()
plt.title('Radius vs Time')

plt.subplot(222)
plt.plot(t, v_vv, label='Velocity Verlet')
plt.plot(t, v_beeman, label='Beeman')
plt.xlabel('Time')
plt.ylabel('Velocity')
plt.legend()
plt.title('Velocity vs Time')

plt.subplot(223)
plt.plot(t, E_vv, label='Velocity Verlet')
plt.plot(t, E_beeman, label='Beeman')
plt.xlabel('Time')
plt.ylabel('Total Energy')
plt.legend()
plt.title('Total Energy vs Time')

plt.subplot(224)
plt.plot(t, (E_vv - E_vv[0])/E_vv[0], label='Velocity Verlet')
plt.plot(t, (E_beeman - E_beeman[0])/E_beeman[0], label='Beeman')
plt.xlabel('Time')
plt.ylabel('Relative Energy Error')
plt.legend()
plt.title('Relative Energy Error vs Time')

plt.tight_layout()


# Print energy conservation statistics
print("Velocity Verlet:")
print(f"Maximum relative error in total energy: {np.max(np.abs((E_vv - E_vv[0])/E_vv[0])):.2e}")
print(f"Standard deviation of relative error in total energy: {np.std((E_vv - E_vv[0])/E_vv[0]):.2e}")

print("\nBeeman:")
print(f"Maximum relative error in total energy: {np.max(np.abs((E_beeman - E_beeman[0])/E_beeman[0])):.2e}")
print(f"Standard deviation of relative error in total energy: {np.std((E_beeman - E_beeman[0])/E_beeman[0]):.2e}")


In [ ]:
x = np.sum(results['e_tot'], axis=1)
plt.plot(results['t'], (x-x[0])/x[0])
plt.xlim(0,0.02)
plt.ylim(-0.01, 0)

In [ ]:
def r(theta):
    return 1-np.cos(theta)

def t(theta):
    return theta - np.sin(theta)

def v(rvals, tvals):
    return np.gradient(rvals, tvals)

def a(vvals, tvals):
    return np.gradient(vvals, tvals)

def E(rvals, vvals, avals):
    # Calculate energy E
    return 0.5 * (vvals**2 - avals * rvals)

theta = np.linspace(0, 4*np.pi, 1000)
rvals, tvals = r(theta), t(theta)
vvals = v(rvals, tvals)
avals = a(vvals, tvals)
Evals = E(rvals, vvals, avals)

plt.figure(figsize=(24, 5))
plt.subplot(141)
plt.plot(tvals, rvals)
plt.xlabel('t')
plt.ylabel('r')
plt.title('r vs t')

plt.subplot(142)
plt.plot(tvals, vvals)
plt.xlabel('t')
plt.ylabel('v')
plt.title('v vs t')

plt.subplot(143)
plt.plot(tvals, avals)
plt.xlabel('t')
plt.ylabel('a')
plt.title('a vs t')

plt.subplot(144)
plt.plot(tvals, Evals)
plt.xlabel('t')
plt.ylabel('E')
plt.title('E vs t')

plt.tight_layout()


In [ ]:
def poorly_formatted_function(   x,y   ):
    return  x+y

if True:
 print(   "This is not properly indented"   )

list_comp = [   x    for x in range(10)   if x % 2 == 0]
